In [9]:
import pandas as pd
import numpy as np
from scipy.stats import linregress

from datetime import datetime
from dateutil.relativedelta import relativedelta

import time

# Lags

In [10]:
def dame_lags_y_deltas(df,nombre_col_original,cant_lags,incluir_deltas=False):
    """
    Agregar columnas con lags y (opcionalmente) deltas
    
    Input:
    - df : (dataframe) Es donde vive la columna nombre_col_original. Ademas tiene q incluir los campos: "product_id" y "periodo"
    - nombre_col_original: (string) Es la columna de interes (ej: "tn")
    - cant_lags: (int) Entero que define la cantidad de lags q va a calcular
    - incluir_deltas: (bool) Se usa para definir si se van a calcular los deltas (True=calcula deltas)
 
    Output:
    - df: (dataframe) Tiene insertadas las nuevas columnas
 
    Ejemplo:
    df = dame_lags_y_deltas(df,nombre_col_original="tn",cant_lags=12,incluir_deltas=True)

    """
    
    # Los datos tienen q estar ordenados por "product_id" y por "periodo"
    df = df.sort_values(by=["product_id","periodo"]).reset_index(drop=True)
    
    #Calculamos los lags
    nombres_columnas_lags = ["tn"]
    for i in range(1,cant_lags+1):
        nombre_nueva_columna = nombre_col_original+"_lag_"+str(i)
        nombres_columnas_lags.append(nombre_nueva_columna)
        df_aux = df.groupby(['product_id'])[nombre_col_original].shift(i).rename(nombre_nueva_columna)
        df=pd.concat([df,df_aux],axis=1)  
        
    #Si corresponde, calculamos los delta lags
    if incluir_deltas == True:
        prefijo_delta_lags = "delta_lag_" # va a tener ESTO mas un numero de 1 a N
        df_aux=pd.DataFrame()
        #Se calculan lag1-lag2, lag2-lag3, lag3-lag4, etc...
        for i in range(0,len(nombres_columnas_lags)-1):
            df_aux[prefijo_delta_lags+str(i+1)] = df[nombres_columnas_lags[i]]-df[nombres_columnas_lags[i+1]]

        #Lo unificamos con el df original
        df = pd.concat([df,df_aux],axis=1)  
        
    return df.copy()

In [11]:
# nombre_col_original = "tn" # columna sobre la cual queremos calcular los lags
# cant_lags = 12

# df = dame_lags_y_deltas(df,nombre_col_original,cant_lags,incluir_deltas=True)

In [12]:
# #Vemos como quedo todo
# with pd.option_context('display.max_columns', None):
#     display(df.head(50))

# Medias moviles

In [13]:
def dame_medias_moviles(df,nombre_col_original,lista_medias_target,incluye_periodo_actual=False):
    
    """
    Calculamos las medias de los ultimos N meses.
    Cada entero "k" que agregues al vector de aca abajo va a crear una columna nueva con 
    el promedio de esos "k" meses previos 
    Podes controlar si incluye o no al periodo actual mediante la variable incluye_periodo_actual
    
    Input: 
    - df : (dataframe) Es donde vive la columna nombre_col_original. Ademas tiene q incluir los campos: "product_id" y "periodo"
    - nombre_col_original: (string) Es la columna de interes (ej: "tn")
    - lista_medias_target: (list) Lista de enteros para saber el tamanio de las ventanas para las medias moviles
    - incluye_periodo_actual: (bool) Es para definir si se debe tener en cuenta o no el periodo actual para el calculo de la media movil (por defecto es False, asi que NO se usa el periodo actual)
    
    Output:
    - df: (dataframe) Tiene insertadas las nuevas columnas
    
    Ejemplo:
    df = dame_medias_moviles(df,nombre_col_original="tn",lista_medias_target=[1,3,6,12],incluye_periodo_actual=False)
    
    """
    
    # Los datos tienen q estar ordenados por "product_id" y por "periodo"
    df = df.sort_values(by=["product_id","periodo"]).reset_index(drop=True)
    
    prefijo = nombre_col_original + "_media_movil_"
    
    if incluye_periodo_actual==True:
        SHIFT=0
    else:
        SHIFT=1

    for i in lista_medias_target:
        df_aux = df.groupby('product_id').rolling(window=i,min_periods=i)[nombre_col_original].mean().shift(SHIFT).rename(prefijo+str(i)).reset_index(drop=True)
        df = pd.concat([df,df_aux],axis=1)
        
    return df.copy()

In [14]:
# lista_medias_target = [2,3,4,6,10] # cant periodos, en meses, que debe medir de largo la ventana
# incluye_periodo_actual = False  # definir si la ventana debe incluir o no al periodo actual
# nombre_col_original = "tn"

# df = dame_medias_moviles(df,nombre_col_original,lista_medias_target,incluye_periodo_actual)

In [15]:
# #Vemos como quedo todo
# with pd.option_context('display.max_columns', None): 
#     display(df.head(50))

# Tendencias



In [16]:
def dame_tendencias(df,nombre_col_original,lista_tamanio_ventana_tendencias,incluye_periodo_actual=False):
    """
    Calcula la pendiente de la recta q aproxima a los elementos dentro de una ventana movil de periodos.
   
   Input:
    - df : (dataframe) Es donde vive la columna nombre_col_original. Ademas tiene q incluir los campos: "product_id" y "periodo"
    - nombre_col_original: (string) Es la columna de interes (ej: "tn")
    - lista_tamanio_ventana_tendencias: (list) Lista de enteros para indicar cant de periodos, en meses, que debe medir de largo la ventana
    - incluye_periodo_actual: (bool) Es para definir si la ventana debe incluir o no al periodo actual. Por defecto vale False (asi q NO lo incluye)

    Output:
    - df: (dataframe) Tiene insertadas las nuevas columnas
    
    Ejemplo:
    df = dame_tendencias(df,nombre_col_original="tn", lista_tamanio_ventana_tendencias=[3,6,12],incluye_periodo_actual=False)
    """
    
    prefijo = nombre_col_original + "_tendencia_"

    # Los datos tienen q estar ordenados por "product_id" y por "periodo"
    df = df.sort_values(by=["product_id","periodo"]).reset_index(drop=True)
    
    if incluye_periodo_actual==True:
        SHIFT=0
    else:
        SHIFT=1

    for i in lista_tamanio_ventana_tendencias:
        df_aux = df.groupby('product_id')[nombre_col_original].rolling(window=i, min_periods=i).apply(lambda v: linregress(np.arange(len(v)), v).slope).shift(SHIFT).rename(prefijo+str(i)).reset_index(level=0, drop=True)
        df = pd.concat([df,df_aux],axis=1)
        
    return df.copy()

In [17]:
# lista_tamanio_ventana_tendencias = [3,6,12] # cant de periodos, en meses, que debe medir de largo la ventana
# incluye_periodo_actual = False # definir si la ventana debe incluir o no al periodo actual
# nombre_col_original="tn"

# df = dame_tendencias(df,nombre_col_original, lista_tamanio_ventana_tendencias,incluye_periodo_actual)

In [18]:
# #Vemos como quedo todo
# with pd.option_context('display.max_columns', None): 
#     display(df.head(50))

# Ventas N meses atras

NOTA: Este lo voy a sacar, porq entiendo q tiene la misma info q los LAGS


In [19]:
# def dame_ventas_n_meses_atras(df,nombre_col_original,lista_ventas_n_meses_atras):
#     """
#     Calculo de las ventas realizadas hace N meses atras  
    
#     Input:
#     - df : (dataframe) Es donde vive la columna nombre_col_original. Ademas tiene q incluir los campos: "product_id" y "periodo"
#     - nombre_col_original: (string) Es la columna de interes (ej: "tn")
#     - lista_ventas_n_meses_atras: (list) Lista de enteros donde especificamos cuales meses hacia atras queremos tener como referencia
   
#     Output:
#     - df: (dataframe) Tiene insertadas las nuevas columnas
    
#     Ejemplo:
#     df = dame_ventas_n_meses_atras(df,nombre_col_original="tn",lista_ventas_n_meses_atras=[1,2,4,8])
#     """
    
#     prefijo = nombre_col_original + "_venta_pasado_" #Para modificar el prefijo de la columna nueva
    
#     # Los datos tienen q estar ordenados por "product_id" y por "periodo"
#     df = df.sort_values(by=["product_id","periodo"]).reset_index(drop=True)
    
#     for i in lista_ventas_n_meses_atras:
#         df_aux = df.groupby("product_id")[nombre_col_original].shift(i).rename(prefijo+str(i))
#         #display(df_aux)
#         df = pd.concat([df,df_aux],axis=1)
        
#     return df.copy()

In [20]:
# lista_ventas_n_meses_atras = [1,2,5,10] #Especificamos a cuales meses hacia atras queremos tener como referencia

# df = dame_ventas_n_meses_atras(df,nombre_col_original,lista_ventas_n_meses_atras)

In [21]:
# #Vemos como quedo todo
# with pd.option_context('display.max_columns', None): 
#     display(df.head(50))
#     #display(df.tail(50))

### Estacionalidad: nro de MES de cada registro [UPDATED!]

In [22]:
def dame_estacionalidad(df):

    """
    Inserta en el df una columna q indica el mes del anio al que corresponde el registro
    
    Input:
    - df: (dataframe) Debe contener el campo "periodo".
    
    Output:
    - df: (dataframe) Tiene insertadas las nuevas columnas
    
    Ejemplo: 
    df = dame_estacionalidad(df)
    """
    
    nombre_nueva_columna = "mes_del_anio"
    
    df_aux=pd.DataFrame()
    df_aux["mes_del_anio"] = df["periodo"]%100

    return pd.concat([df,df_aux],axis=1).copy()

In [23]:
# df = dame_estacionalidad(df)
# #df

In [24]:
# #Vemos como quedo todo
# with pd.option_context('display.max_columns', None): 
#     display(df.head(50))
#     #display(df.tail(50))

### Tn de los productos de igual marca, pero tamanio inmediatamente mayor y menor

In [25]:
def dame_hermanos_mayores_y_menores(df, nombre_col_original, periodos_previos):

    """
    Requiere tener instalada la biblioteca empresa4.
    
    Se genera información de ventas (tn) de productos de igual categoria 1,2 y 3 e igual marca, pero distinto tamanio.
    Llamamos Hermano Mayor a un producto de este tipo, cuando es mas grande que otro.
    Llamamos Hermano Menor a un producto de este tipo, cuando es mas chico que otro.
    
    Esta funcion permite incorporar columnas con info de ventas de los hermanos mayores y menores para N periodos en el pasado.
    
    Input:
    * df : (dataframe) Es donde vive la columna nombre_col_original. Ademas tiene q incluir los campos: "product_id" y "periodo"
    * nombre_col_original: (string) Es la columna de referencia (por ejemplo "tn")
    * periodos_previos: (list) Es una lista de enteros para indicar los periodos sobre los que quiero traer info de ventas de hermanos mayores y menores
    
    Output:
    - df: (dataframe) Tiene insertadas las nuevas columnas de toneladas vendidas, pero tambien el product_id de los hermanos mayor y menor
                      Nota: "hermano_M" se refiere a "hermano mayor" mientras que "hermano_m" se refiere a "hermano menor".
                    
    Ejemplo:
    df = dame_hermanos_mayores_y_menores(df,nombre_col_original="tn",periodos_previos=[1,2,5])
    
    """    
    
    # En este df tenemostodos los productos (product_id), 
    # con las categorias a las q pertenecen (de nivel 1, 2 y 3)
    df_productos = get_dataset("maestro_productos")

    #Agrupamos por cat1,cat2 y cat3 y brand
    df_productos.sort_values(by=["cat1","cat2","cat3","brand","sku_size"],inplace=True)
    grupos = df_productos.groupby(["cat1","cat2","cat3","brand"])[["product_id"]]

    df_prod_hermanos = pd.DataFrame()
    for grupo in grupos:
        df_aux = grupo[1].copy()
        df_aux ["hermano_mayor"] = grupo[1].shift(-1)
        df_aux ["hermano_menor"] = grupo[1].shift(1)
        df_prod_hermanos = pd.concat([df_prod_hermanos,df_aux])
    # #en este df tenemos cada product_id con info de quienes son los productos hermano_mayor y hermano_menor
    # df_prod_hermanos
    
    # Los datos tienen q estar ordenados por "product_id" y por "periodo"
    df = df.sort_values(by=["product_id","periodo"]).reset_index(drop=True)

    #le agregamos a df la info de quien es hermano_maryo y menor de cada product_id
    df=pd.merge(
        df, #left
        df_prod_hermanos, #right
        how="left",
        on="product_id",
    )

    # Ahora haremos dos cosas:
    # 1) Agarrar un product id en un periodo -> fijate quien es el hno mayor, traeme lo q vendio en N periodos del pasado (sin incluir el actual)
    # 2) Agarrar un product id en un periodo -> fijate quien es el hno menor, traeme lo q vendio en N periodos del pasado (sin incluir el actual)

    acumulado_hermanos_menores = []
    acumulado_hermanos_mayores = []

    for idx,row in df.iterrows():

        #el periodo viene escrito con .0, asi que lo pasamos a int 
        periodo_actual = row["periodo"].astype(np.int64)
        
#         print(periodo_actual)
#         print(type(periodo_actual))
        
        
        hermano_menor = row["hermano_menor"]
        hermano_mayor = row["hermano_mayor"]
        hermano_del_medio = row["product_id"]

        #Levantamos la info de todos los hermanos menores
        if np.isnan(hermano_menor)==False:
            df_aux = df.loc[ df["product_id"] == hermano_menor ]
            acumulado_aux = []
            for i in periodos_previos:
                #convertimos a date, restamos un dia, y volvemos a int:
                periodo_target = pd.to_datetime(str(periodo_actual), format='%Y%m') - relativedelta(months=i)
                periodo_target = int(periodo_target.strftime('%Y%m'))
                aux = df_aux.loc [df_aux["periodo"]==periodo_target][nombre_col_original].to_list()
                if len(aux)==0:
                    aux = [np.NaN]
                acumulado_aux.append(aux)
            acumulado_aux = [n for elemento in acumulado_aux for n in elemento]
        else:
            #Si entra aca, es porque NO tiene hermanos menores
            #asi que las tn de los hermanos menores es todo NAN
            acumulado_aux = [np.nan for j in range(0,len(periodos_previos))]

        acumulado_hermanos_menores.append(acumulado_aux)

        #generamos la info de todos los hermanos mayores
        if np.isnan(hermano_mayor)==False:
            df_aux = df.loc[ df["product_id"] == hermano_mayor ]
            acumulado_aux = []
            for i in periodos_previos:
                #convertimos a date, restamos un dia, y volvemos a int:
                periodo_target = pd.to_datetime(str(periodo_actual), format='%Y%m') - relativedelta(months=i)
                periodo_target = int(periodo_target.strftime('%Y%m'))
                aux = df_aux.loc [df_aux["periodo"]==periodo_target][nombre_col_original].to_list()
                if len(aux)==0:
                    aux = [np.NaN]
                acumulado_aux.append(aux)
            acumulado_aux = [n for elemento in acumulado_aux for n in elemento]
        else:
            #Si entra aca, es porque NO tiene hermanos mayores
            #asi que las tn de los hermanos mayores es todo NAN
            acumulado_aux = [np.nan for j in range(0,len(periodos_previos))]

        acumulado_hermanos_mayores.append(acumulado_aux)    

    #Finalmente, armamos un df con toda la info
    df = pd.concat(
        [
        df,
        pd.DataFrame(acumulado_hermanos_mayores, columns = [nombre_col_original+"_hermano_M_-"+str(periodos_previos[i])   for i in range(0,len(periodos_previos))    ]), 
        pd.DataFrame(acumulado_hermanos_menores, columns = [nombre_col_original+"_hermano_m_-"+str(periodos_previos[i])   for i in range(0,len(periodos_previos))    ])
        ], axis=1
    )
    
    return df.copy()

In [26]:
# periodos_previos = [1,2,5] #periodos sobre los que quiero traer info de ventas de hermanos mayores y menores
# nombre_col_original = "tn"

# df = dame_hermanos_mayores_y_menores(df,nombre_col_original,periodos_previos)

### Insertamos campos de clima 

In [27]:
def dame_info_clima(df):
    
    df_clima = pd.read_csv("df_clima_completo.csv") #este df lo pase por whatsapp, y esta tb subido al compartido

    # hacemos q el df_clima se vuelve "horizontal", es decir, por cada periodo necesito q 
    # cada lugar sea una columna

    df_clima.sort_values(by=["ciudad","periodo"],inplace=True)
    grupos = df_clima.groupby("periodo")

    periodos = list(set(df_clima["periodo"].to_list()))
    periodos.sort()

    columnas=df_clima.columns[2:]
    ciudades = list(set(df_clima.ciudad.to_list()))
    ciudades.sort()
    nombres_nuevas_columnas = [ciudad+"_"+columna for ciudad in ciudades for columna in columnas]

    df_clima_transformado = pd.DataFrame(columns = nombres_nuevas_columnas)
    for grupo in grupos:
        periodo = grupo[0]
        grupo[1].drop("periodo",inplace=True,axis=1)

        datos = []

        #print(grupo[1])
        for idx,row in grupo[1].iterrows():
            datos.append(row["temp_max"])
            datos.append(row["humedad_max"])
            datos.append(row["presion_max"])
            datos.append(row["temp_min"])
            datos.append(row["humedad_min"])
            datos.append(row["presion_min"])
            datos.append(row["temp_promedio"])
            datos.append(row["humedad_promedio"])
            datos.append(row["presion_promedio"])


        #datos = [dato for lista in datos for dato in lista]

        df_clima_transformado.loc[len(df_clima_transformado)] = datos

    #le ponemos los promedios, y ponemos esa col primera
    df_clima_transformado["periodo"] = periodos
    temp_cols=df_clima_transformado.columns.tolist()
    new_cols=temp_cols[-1:] + temp_cols[0:-1]
    df_clima_transformado=df_clima_transformado[new_cols]

    df = pd.merge(df, df_clima_transformado, how="left", on="periodo")
    
    return df

### Indicador de CATASTROFE 

NOTA: Este habría q repensarlo... asi q por ahora lo dejo comentado

In [28]:
# def dame_catastrofe(df):
    
#     """
#     # Nos interesa monitorear el campo "tn", q mide las toneladas ENTREGADAS
#     # Nos interesa ver dos cosas:
#     # - Que hubo pedido, pero no se entrego nada (cust_request_tn!=0 pero tn==0) (ESTO AL FINAL NO LO TUVE EN CUENTA)
#     # - Que el mes inmediatamente anterior y el inmediatamente posterior, hubo entregas, pero ese no (tn==0, pero tn-1!= y tn+1!=0) 
    
#     Input: 
#     - df (tiene que incluir la columna "tn")
    
#     Output:
#     - el df con una nueva columna "catastrofe", donde 1 indica q ese mes hubo algun evento inesperado q hizo q la produccion sea CERO
    
#     Ejemplo de uso:
#     df = dame_catastrofe(df)
    
#     """
    
#     df["tn_-1"] = df["tn"].shift(+1)
#     df["tn_+1"] = df["tn"].shift(-1)

#     #df.loc[ (df["cust_request_tn"]!=0) & (df["tn"]==0) & (df["tn_-1"]!=0) & (df["tn_+1"]!=0) ,"catastrofe"]=1
#     df.loc[ (df["tn"]==0) & (df["tn_-1"]!=0) & (df["tn_+1"]!=0) ,"catastrofe"]=1
   
    
#     df.drop("tn_-1",axis=1,inplace=True)
#     df.drop("tn_+1",axis=1,inplace=True)

#     df["catastrofe"] = df["catastrofe"].fillna(0)
    
#     return df

### Valor del punto MENOS la media movil

In [29]:
def dame_dalmm(df, nombre_col_original):
    """
    Calcula la distancia entre "tn" y la media movil, para tantas medias moviles como haya disponibles.
    Si no hay medias moviles disponibles, devuelve un df q solo contiene "periodo" y "product_id"
    
    Input:
    - nombre_col_original: "tn"
    
    Output: un df q solo tiene
    """
    
    #Busca a ver si hay alguna columna de las disponibles tiene la palabra "_media_movil_"
    columnas_m_m = [i for i in df.columns if "_media_movil" in i]

    df_aux=pd.DataFrame(df[["periodo","product_id"]])
    for columna in columnas_m_m:
        df_aux["dalmm_"+columna] = df[nombre_col_original]-df[columna] 

    return df_aux

### Stocks [Nuevo!]

In [30]:
def dame_stocks(df):
    
    df_stocks = pd.read_csv("../tb_stocks_02_productos_todos.csv")
    df = pd.merge(df, df_stocks, how="left", on=["periodo","product_id"])
    
    return df

# Funciones del FE Seguro

In [31]:
def dame_campos_nuevos(df_mes_actual, df_meses_anteriores,configs):
    """
    Esta funcion recibe 2 df: Uno del mes actual, y otro con info de todos los meses anteriores
    Se encarga de dos cosas: 
    - Crear nuevos campos para el mes actual, usando solo el df del mes actual
    - Crear nuevos campos para los meses pasados, usando solo los datos de meses pasados
    
    Una vez hecho eso, incorpora todas las nuevas columnas AL MES ACTUAL.
    De esta forma, nos aseguramos que NUNCA usa datos del futuro para crear campos nuevos.
    """
    
    #recuperamos el periodo actual
    periodo_actual = list(set(df_mes_actual["periodo"].to_list()))[0]
    
    campo_FE = configs["campo_FE"] 
    
    ##############
    # Aca va todo el FE que quieras hacer sobre los campos del mes ACTUAL (nada de usar datos de meses anteriores)
    # ...quitar columnas del mes actual
    # ...agregar columnas del mes actual
    # ...tranasformar columnas del mes actual
    ##############

    df_materia_prima_presente = df_mes_actual[["periodo","product_id"]]
    lista_df_FE_acumulado = []

    if configs["estacionalidad"]:
        df_aux = dame_estacionalidad(df_materia_prima_presente)
        lista_df_FE_acumulado.append(df_aux)
    
    if configs["clima"]:
        df_aux = dame_info_clima(df_materia_prima_presente)
        lista_df_FE_acumulado.append(df_aux)

#     df_aux = pd.get_dummies(df_mes_actual[["periodo","product_id","product_category"]], columns = ["product_category"])
#     lista_df_FE_acumulado.append(df_aux)
    
#     for item in lista_df_FE_acumulado:
#         display(item)
    
    ###############
    #Una vez q terminaste con el FE del mes actual, los unis
    ###############
    df_mes_actual_FE = df_materia_prima_presente.copy()
    for mini_df in lista_df_FE_acumulado:
        # JOIN FINAL de las nuevas columnas calculadas en base al mes actual (df_mes_actual_FE)
        # y de las nuevas columnas calculadas en base a meses anteriores (df_meses_anteriores_FE)
        df_mes_actual_FE = pd.merge(df_mes_actual_FE, #left
                mini_df, #right
                how="left",
                on=["periodo","product_id"],
                )
    ##############
    
    
    ##############
    # Aca va todo el FE que quieras hacer sobre los campos de MESES ANTERIORES
    ##############   
    #FE para crear nuevos campos en base a meses ANTERIORES (la materia prima)
    df_materia_prima_pasado = pd.concat([df_mes_actual,df_meses_anteriores])
    df_materia_prima_pasado = df_materia_prima_pasado[["periodo","product_id",campo_FE]]
    lista_df_FE_acumulado = []
        
    if configs["lags"]:
        #Lags
        cant_lags=configs["cant_lags"]
        incluir_deltas=configs["deltas"]
        df_aux = dame_lags_y_deltas(df_materia_prima_pasado,
                                    nombre_col_original=campo_FE,
                                    cant_lags=cant_lags,
                                    incluir_deltas=incluir_deltas)
        #De lo q hayas calculado usando info de meses anteriores, SOLO ME INTERESAN las col de mes actual
        lista_df_FE_acumulado.append(df_aux.loc[ df_aux["periodo"]==periodo_actual].drop(campo_FE,axis=1))
        #display(df_aux)


    if configs["medias_moviles"]:
        #medias moviles
        #lista_medias_target = [1,2]
        lista_medias_target = configs["tamanio_ventanas_m_m"]
        incluye_periodo_actual = True
        #display(df_materia_prima_pasado)
        df_aux = dame_medias_moviles(df_materia_prima_pasado,
                            nombre_col_original=campo_FE,
                            lista_medias_target=lista_medias_target,
                            incluye_periodo_actual=incluye_periodo_actual)
        #De lo q hayas calculado usando info de meses anteriores, SOLO ME INTERESAN las col de mes actual
        lista_df_FE_acumulado.append(df_aux.loc[ df_aux["periodo"]==periodo_actual].drop(campo_FE,axis=1))
    

    if configs["dalmm"]:
        #distancia a la media movil (calcula de nuevo las MM y dps la distancia entre "tn" y esas medias moviles)
        lista_medias_target = configs["tamanio_ventanas_m_m"]
        incluye_periodo_actual = True
        df_aux = dame_medias_moviles(df_materia_prima_pasado,
                            nombre_col_original=campo_FE,
                            lista_medias_target=lista_medias_target,
                            incluye_periodo_actual=incluye_periodo_actual)
        df_aux = dame_dalmm(df=df_aux, nombre_col_original=campo_FE)
        #De lo q hayas calculado usando info de meses anteriores, SOLO ME INTERESAN las col de mes actual
        lista_df_FE_acumulado.append(df_aux.loc[df_aux["periodo"]==periodo_actual])
        #display(df_aux)

    if configs["tendencias"]:
        #tendencias
        #lista_tamanio_ventana_tendencias = [i for i in range(1,2)] # cant de periodos, en meses, que debe medir de largo la ventana
        lista_tamanio_ventana_tendencias = configs["tamanio_ventanas_tendencias"]
        incluye_periodo_actual = True # definir si la ventana debe incluir o no al periodo actual
        df_aux = dame_tendencias(df_materia_prima_pasado,
                                 nombre_col_original=campo_FE,
                                 lista_tamanio_ventana_tendencias=lista_tamanio_ventana_tendencias,
                                 incluye_periodo_actual=incluye_periodo_actual)
        #De lo q hayas calculado usando info de meses anteriores, SOLO ME INTERESAN las col de mes actual
        lista_df_FE_acumulado.append(df_aux.loc[ df_aux["periodo"]==periodo_actual].drop(campo_FE,axis=1))

    
    if configs["hermanos_mayores_y_menores"]:
        #productos hermanos mayores y menores
        #periodos_previos = [i for i in range(1,3)] #periodos sobre los que quiero traer info de ventas de hermanos mayores y menores
        periodos_previos = [1]
        df = dame_hermanos_mayores_y_menores(df_materia_prima_pasado,
                                             nombre_col_original=campo_FE,
                                             periodos_previos=periodos_previos)
    
    
    ###############
    #Una vez q terminaste con el FE, unis todo y armas el output
    ###############
    df_output = df_mes_actual_FE.copy()
    for mini_df in lista_df_FE_acumulado:
        # JOIN FINAL de las nuevas columnas calculadas en base al mes actual (df_mes_actual_FE)
        # y de las nuevas columnas calculadas en base a meses anteriores (df_meses_anteriores_FE)
        df_output = pd.merge(df_output, #left
                mini_df, #right
                how="left",
                on=["periodo","product_id"],
                )
    
    return df_output

In [32]:
def dame_FE_seguro(df_al_que_calcularle_FE,df_con_todos_los_periodos,configs):
    
    """
    Esta funcion hace FE para CADA mes, usando datos del mes en curso y datos de los meses previos (de forma separada).
    
    Input:
    - df_al_que_calcularle_FE (el df con los periodos q queres calcular FE), el cual 
                              vas a usar para ir sacando "el periodo actual", tantos como contenga
    - df_con_todos_los_periodos (el general), lo vas a usar para sacar info de "los periodos anteriores"
    - configs: diccionario dnd esta definido que FE se va a ejecutar (lags? deltas? medias? etc)
    
    Output: 
    - El df con nuevos campos creados de forma segura.
    """

    #Todos los periodos disponibles
    periodos_todos = list(set(df_con_todos_los_periodos["periodo"].to_list()))
    periodos_todos.sort(reverse=True) #ordenados de forma desc: ... 201705, 201704, 201703, ...

    #Los periodos sobre los q queremos hacer FE (puede ser uno o muchos)
    periodos_FE = list(set(df_al_que_calcularle_FE["periodo"].to_list()))
    periodos_FE.sort(reverse=True) #Si son muchos, quedan ordenados de forma desc: ... 201705, 201704, 201703, ...

    df_campos_nuevos_acumulados = pd.DataFrame()

    for periodo in periodos_FE:

            periodo_actual = periodo #int
            periodos_anteriores = [p for p in periodos_todos if p<periodo_actual] #lista
            
            #print(periodos_anteriores)

            df_mes_actual = df_al_que_calcularle_FE.loc[ df_al_que_calcularle_FE["periodo"] == periodo_actual ]
            df_meses_anteriores = df_con_todos_los_periodos.loc[ df_con_todos_los_periodos["periodo"].isin(periodos_anteriores) ]                             

#             print(df_al_que_calcularle_FE)
#             print("*********************")
#             print(df_meses_anteriores)
            
            #esto devuelve el FE para "periodo_actual". Hay q irlo acumulando
            #para tantos periodos haya en periodos_target, y devolverlo al final.
            df_aux = dame_campos_nuevos(df_mes_actual, df_meses_anteriores,configs) #LINEA ORIIGNAL
                        
            df_campos_nuevos_acumulados = pd.concat([df_campos_nuevos_acumulados,df_aux])

    df_campos_nuevos_acumulados.sort_values(by=["product_id","periodo"],inplace=True)

    #En este ultimo paso, unificamos: el df inicial con las nuevas col construidas
    df_FE = pd.merge(df_al_que_calcularle_FE, #left
                   df_campos_nuevos_acumulados, #right
                   how="left", on=["periodo","product_id"],)
    
    return df_FE

## Configuracion del FE seguro

In [33]:
#CONFIGURACION FE:
config_FE = {
            "campo_FE":"tn", #campo al que le queres aplicar lags, deltas, medias moviles, etc
            "estacionalidad":True, #genera una columna con el nro de mes actual
            "clima":True, #incorpora columnas con info del clima para el mes actual
            "stocks":True, #este no se bien que representa, ¿los stocks q sobraron al final de cada periodo?
            "lags":True, # True para que calcule lags
            "cant_lags":18, #cant de lags q va a calcular
            "deltas":True, #True para que calcule deltas (tantos como pueda)
            "medias_moviles":True, #True para q calcule medias moviles
            "tamanio_ventanas_m_m": [3,6], #cada elemento de la lista es un tamanio de ventana
            "dalmm": True, # calcula (o no) la distancia a las medias moviles q haya disponibles
            "tendencias": True, #True para q calcule tendencias
            "tamanio_ventanas_tendencias":[6,12], 
            "hermanos_mayores_y_menores":True, #True para q incorpore columnas con info de tn de compra de prod hermanos
            "periodos_previos_hermanos_M_y_m": [1,6], #cada elemento de la lista es un periodo anterior del cual va a traer info 
}

## Ejecución del FE Seguro

In [4]:
import pandas as pd
import requests
from io import StringIO

url='https://drive.google.com/file/d/1xwJmomvqSsDARENHRdfI4EmIt6aXFX2x/view?usp=sharing'
file_id = url.split('/')[-2]
dwn_url='https://drive.usercontent.google.com/download?id=' + file_id + '&export=download&authuser=0&confirm=t&uuid=95f36248-0659-4956-adaa-bd1e0564383f&at=APZUnTX9peDngh8O7LsXKmRoDw12:1701132306870'
url2 = requests.get(dwn_url).text
csv_raw = StringIO(url2)
df = pd.read_csv(csv_raw)
df.head()

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,cat3,brand,sku_size,plan_precios_cuidados
0,201701,10001,20480,6.0,0.85598,0.85598,PC,CABELLO,ACONDICIONADOR,SHAMPOO1,750.0,0
1,201701,10001,20482,1.0,0.75874,0.75874,PC,CABELLO,SHAMPOO,CAPILATIS,400.0,0
2,201701,10001,20483,0.0,0.00000,0.00000,PC,CABELLO,SHAMPOO,SHAMPOO3,10.0,0
3,201701,10001,20484,6.0,0.89664,0.89664,PC,DEOS,Aero,DEOS3,100.0,0
4,201701,10001,20485,2.0,0.73612,0.73612,HC,VAJILLA,Gel,MUSCULO,300.0,0


In [5]:
df

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,cat3,brand,sku_size,plan_precios_cuidados
0,201701,10001,20480,6.0,0.85598,0.85598,PC,CABELLO,ACONDICIONADOR,SHAMPOO1,750.0,0
1,201701,10001,20482,1.0,0.75874,0.75874,PC,CABELLO,SHAMPOO,CAPILATIS,400.0,0
2,201701,10001,20483,0.0,0.00000,0.00000,PC,CABELLO,SHAMPOO,SHAMPOO3,10.0,0
3,201701,10001,20484,6.0,0.89664,0.89664,PC,DEOS,Aero,DEOS3,100.0,0
4,201701,10001,20485,2.0,0.73612,0.73612,HC,VAJILLA,Gel,MUSCULO,300.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
19109368,201912,10582,20474,0.0,0.00000,0.00000,PC,CABELLO,ACONDICIONADOR,SHAMPOO1,400.0,0
19109369,201912,10582,20476,0.0,0.00000,0.00000,PC,PIEL2,Jabon Barra,SHAMPOO2,125.0,0
19109370,201912,10582,20477,0.0,0.00000,0.00000,HC,VAJILLA,Gel,MUSCULO,300.0,0
19109371,201912,10582,20478,0.0,0.00000,0.00000,HC,HOGAR,PISOS,MUSCULO,750.0,0


In [6]:
#Ordenamos todo el df por product_id y periodo (esto es CLAVE para todo lo q sigue)
df = df.sort_values(by=["product_id", "customer_id", "periodo"]).reset_index(drop=True)

In [34]:
#Calcular el FE sobre CADA PERIODO de TODO EL DATASET

#obtenemos la lista de todos los periodos:
lista_todos_los_periodos = list(set(df["periodo"].to_list()))
lista_todos_los_periodos.sort()

start = time.time()

df_FE_completo = pd.DataFrame()
for periodo in lista_todos_los_periodos:
    df_aux =  df.loc[df["periodo"]==periodo]
    df_FE_parcial = dame_FE_seguro(df_aux,df,config_FE)
    df_FE_completo = pd.concat([df_FE_completo,df_FE_parcial])

df_FE_completo.reset_index(drop=True, inplace=True)

end = time.time()    
print(end - start)

: 

In [30]:
#definimos el campo CLASE 
y = df_FE_completo.groupby(['product_id'])["tn"].shift(-2).rename("CLASE")

In [31]:
#Armamos el dataset completo final, con clase 
df_FE_completo_con_CLASE = pd.concat([df_FE_completo,y],axis=1)  

In [33]:
df_FE_completo_con_CLASE

,periodo,product_id,cust_request_qty,cust_request_tn,tn,mes_del_anio,bariloche_temp_max,bariloche_humedad_max,bariloche_presion_max,bariloche_temp_min,...,delta_lag_16,delta_lag_17,delta_lag_18,tn_media_movil_3,tn_media_movil_6,dalmm_tn_media_movil_3,dalmm_tn_media_movil_6,tn_tendencia_6,tn_tendencia_12,CLASE
0,201701,20001,479.0,937.72717,934.77222,1,32.0,97.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1303.35771
1296,201702,20001,432.0,833.72187,798.01620,2,33.0,91.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1069.96130
2592,201703,20001,509.0,1330.74697,1303.35771,3,27.0,95.0,NaN,0.0,...,NaN,NaN,NaN,1012.048710,NaN,291.309000,NaN,NaN,NaN,1502.20132
3888,201704,20001,279.0,1132.94430,1069.96130,4,20.0,97.0,NaN,2.0,...,NaN,NaN,NaN,1057.111737,NaN,12.849563,NaN,NaN,NaN,1520.06539
5184,201705,20001,701.0,1550.68936,1502.20132,5,17.0,98.0,NaN,-6.0,...,NaN,NaN,NaN,1291.840110,NaN,210.361210,NaN,NaN,NaN,1030.67391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41471,201908,21299,0.0,0.00000,0.00000,8,14.0,98.0,NaN,-6.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000
42767,201909,21299,0.0,0.00000,0.00000,9,22.0,97.0,NaN,-7.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000
44063,201910,21299,0.0,0.00000,0.00000,10,24.0,97.0,NaN,-6.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000
45359,201911,21299,0.0,0.00000,0.00000,11,26.0,91.0,NaN,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,NaN


In [32]:
#Vemos como quedo todo
df_FE_completo_con_CLASE = df_FE_completo_con_CLASE.sort_values(by=["product_id","periodo"])
with pd.option_context('display.max_columns', None): 
    display(df_FE_completo_con_CLASE.head(50))

,periodo,product_id,cust_request_qty,cust_request_tn,tn,mes_del_anio,bariloche_temp_max,bariloche_humedad_max,bariloche_presion_max,bariloche_temp_min,bariloche_humedad_min,bariloche_presion_min,bariloche_temp_promedio,bariloche_humedad_promedio,bariloche_presion_promedio,bragado_temp_max,bragado_humedad_max,bragado_presion_max,bragado_temp_min,bragado_humedad_min,bragado_presion_min,bragado_temp_promedio,bragado_humedad_promedio,bragado_presion_promedio,buenos-aires_temp_max,buenos-aires_humedad_max,buenos-aires_presion_max,buenos-aires_temp_min,buenos-aires_humedad_min,buenos-aires_presion_min,buenos-aires_temp_promedio,buenos-aires_humedad_promedio,buenos-aires_presion_promedio,comodoro-rivadavia_temp_max,comodoro-rivadavia_humedad_max,comodoro-rivadavia_presion_max,comodoro-rivadavia_temp_min,comodoro-rivadavia_humedad_min,comodoro-rivadavia_presion_min,comodoro-rivadavia_temp_promedio,comodoro-rivadavia_humedad_promedio,comodoro-rivadavia_presion_promedio,cordoba_temp_max,cordoba_humedad_max,cordoba_presion_max,cordoba_temp_min,cordoba_humedad_min,cordoba_presion_min,cordoba_temp_promedio,cordoba_humedad_promedio,cordoba_presion_promedio,corrientes_temp_max,corrientes_humedad_max,corrientes_presion_max,corrientes_temp_min,corrientes_humedad_min,corrientes_presion_min,corrientes_temp_promedio,corrientes_humedad_promedio,corrientes_presion_promedio,esquel_temp_max,esquel_humedad_max,esquel_presion_max,esquel_temp_min,esquel_humedad_min,esquel_presion_min,esquel_temp_promedio,esquel_humedad_promedio,esquel_presion_promedio,formosa_temp_max,formosa_humedad_max,formosa_presion_max,formosa_temp_min,formosa_humedad_min,formosa_presion_min,formosa_temp_promedio,formosa_humedad_promedio,formosa_presion_promedio,gualeguaychu_temp_max,gualeguaychu_humedad_max,gualeguaychu_presion_max,gualeguaychu_temp_min,gualeguaychu_humedad_min,gualeguaychu_presion_min,gualeguaychu_temp_promedio,gualeguaychu_humedad_promedio,gualeguaychu_presion_promedio,jujuy_temp_max,jujuy_humedad_max,jujuy_presion_max,jujuy_temp_min,jujuy_humedad_min,jujuy_presion_min,jujuy_temp_promedio,jujuy_humedad_promedio,jujuy_presion_promedio,junin_temp_max,junin_humedad_max,junin_presion_max,junin_temp_min,junin_humedad_min,junin_presion_min,junin_temp_promedio,junin_humedad_promedio,junin_presion_promedio,la-plata_temp_max,la-plata_humedad_max,la-plata_presion_max,la-plata_temp_min,la-plata_humedad_min,la-plata_presion_min,la-plata_temp_promedio,la-plata_humedad_promedio,la-plata_presion_promedio,la-rioja_temp_max,la-rioja_humedad_max,la-rioja_presion_max,la-rioja_temp_min,la-rioja_humedad_min,la-rioja_presion_min,la-rioja_temp_promedio,la-rioja_humedad_promedio,la-rioja_presion_promedio,mar-del-plata_temp_max,mar-del-plata_humedad_max,mar-del-plata_presion_max,mar-del-plata_temp_min,mar-del-plata_humedad_min,mar-del-plata_presion_min,mar-del-plata_temp_promedio,mar-del-plata_humedad_promedio,mar-del-plata_presion_promedio,mendoza_temp_max,mendoza_humedad_max,mendoza_presion_max,mendoza_temp_min,mendoza_humedad_min,mendoza_presion_min,mendoza_temp_promedio,mendoza_humedad_promedio,mendoza_presion_promedio,neuquen_temp_max,neuquen_humedad_max,neuquen_presion_max,neuquen_temp_min,neuquen_humedad_min,neuquen_presion_min,neuquen_temp_promedio,neuquen_humedad_promedio,neuquen_presion_promedio,parana_temp_max,parana_humedad_max,parana_presion_max,parana_temp_min,parana_humedad_min,parana_presion_min,parana_temp_promedio,parana_humedad_promedio,parana_presion_promedio,posadas_temp_max,posadas_humedad_max,posadas_presion_max,posadas_temp_min,posadas_humedad_min,posadas_presion_min,posadas_temp_promedio,posadas_humedad_promedio,posadas_presion_promedio,puerto-deseado_temp_max,puerto-deseado_humedad_max,puerto-deseado_presion_max,puerto-deseado_temp_min,puerto-deseado_humedad_min,puerto-deseado_presion_min,puerto-deseado_temp_promedio,puerto-deseado_humedad_promedio,puerto-deseado_presion_promedio,rawson_temp_max,rawson_humedad_max,rawson_presion_max,rawson_temp_min,rawson_humed


# To Do:

1- FE que haga lo siguiente: q me diga CUANTOS MESES HACE Q EXISTE EL PRODUCTO (antiguedad_del_producto, en meses). Para hacer esto vamos a tener q construir un nuevo campo q nos diga cuando es creado un producto.  
2- FE para caracterizar los ceros de la serie temporal